# Dashboard Interativo - Análise de Filmes TMDB
Este dashboard foi construído com a biblioteca `panel` para permitir a visualização dinâmica dos dados do dataset TMDB.

In [1]:
import pandas as pd
import panel as pn
import holoviews as hv
from holoviews import opts
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from helper_functions import *

pn.extension(design='material')
hv.extension('bokeh')

# Carregar e pré-processar os dados (usando a mesma lógica do tp_final.ipynb)
FILE_PATH = 'TMDB_movie_dataset_v11.csv'
AMOUNT_RECORDS = 300 # Usar o mesmo limite para consistência

try:
    # Nota: A função load_dataset_with_directors faz um request à API, o que pode ser lento.
    # Para o dashboard, vamos carregar o CSV diretamente e fazer o pré-processamento local.
    df = pd.read_csv(FILE_PATH, nrows=AMOUNT_RECORDS)
    
    # Pré-processamento básico
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
    df['release_year'] = df['release_date'].dt.year
    df['revenue'] = df['revenue'].fillna(0)
    df['budget'] = df['budget'].fillna(0)
    df['profit'] = df['revenue'] - df['budget']
    
    def parse_genres(genres_str):
        if pd.isna(genres_str) or genres_str == '':
            return []
        return [genre.strip() for genre in genres_str.split(',')]
    
    df['genres_list'] = df['genres'].apply(parse_genres)
    df['main_genre'] = df['genres_list'].apply(lambda x: x[0] if x else 'Unknown')
    
    # Remover linhas com valores nulos críticos para o dashboard
    df_cleaned = df.dropna(subset=['release_year', 'vote_average', 'vote_count']).copy()
    df_cleaned['release_year'] = df_cleaned['release_year'].astype(int)
    
except FileNotFoundError:
    print(f"Erro: O ficheiro {FILE_PATH} não foi encontrado. Certifique-se de que está no diretório correto.")
    df_cleaned = pd.DataFrame()
except Exception as e:
    print(f"Ocorreu um erro durante o carregamento/pré-processamento dos dados: {e}")
    df_cleaned = pd.DataFrame()

# --------------------------------------------------------------------------------
# Componentes do Dashboard
# --------------------------------------------------------------------------------

# 1. Widgets de Controlo
if not df_cleaned.empty:
    min_year = int(df_cleaned['release_year'].min())
    max_year = int(df_cleaned['release_year'].max())
    
    year_slider = pn.widgets.IntRangeSlider(
        name='Ano de Lançamento',
        start=min_year,
        end=max_year,
        value=(min_year, max_year),
        step=1
    )
    
    genre_select = pn.widgets.MultiChoice(
        name='Género Principal',
        options=list(df_cleaned['main_genre'].unique()),
        value=list(df_cleaned['main_genre'].unique())
    )
    
    min_votes_slider = pn.widgets.IntSlider(
        name='Mínimo de Votos',
        start=0,
        end=int(df_cleaned['vote_count'].max()),
        value=0,
        step=100
    )
    
    # 2. Função de Filtragem (agora sem o decorador @pn.depends)
    def filtered_data(years, genres, min_votes):
        df_filtered = df_cleaned[
            (df_cleaned['release_year'] >= years[0]) &
            (df_cleaned['release_year'] <= years[1]) &
            (df_cleaned['main_genre'].isin(genres)) &
            (df_cleaned['vote_count'] >= min_votes)
        ]
        return df_filtered
    
    # 3. Visualizações Dinâmicas (agora dependem dos widgets e chamam filtered_data)
    
    # Gráfico 1: Distribuição da Média de Votos
    @pn.depends(year_slider.param.value, genre_select.param.value, min_votes_slider.param.value)
    def plot_vote_distribution(years, genres, min_votes):
        df_filtered = filtered_data(years, genres, min_votes)
        if df_filtered.empty:
            return pn.pane.Markdown("**Sem dados para os filtros selecionados.**")
        
        # Usar HoloViews/Bokeh para interatividade
        hist = hv.Histogram(df_filtered['vote_average'], label='Média de Votos').opts(
            title='Distribuição da Média de Votos',
            xlabel='Média de Votos',
            ylabel='Frequência',
            height=300, width=400, tools=['hover']
        )
        return hist
    
    # Gráfico 2: Lucro vs. Média de Votos (Scatter Plot)
    @pn.depends(year_slider.param.value, genre_select.param.value, min_votes_slider.param.value)
    def plot_profit_vs_vote(years, genres, min_votes):
        df_filtered = filtered_data(years, genres, min_votes)
        if df_filtered.empty:
            return pn.pane.Markdown("**Sem dados para os filtros selecionados.**")
        
        # Filtrar para filmes com lucro e orçamento > 0 para melhor visualização
        df_plot = df_filtered[(df_filtered['profit'] > 0) & (df_filtered['budget'] > 0)]
        
        scatter = hv.Scatter(df_plot, kdims=['vote_average'], vdims=['profit', 'title']).opts(
            title='Lucro vs. Média de Votos',
            xlabel='Média de Votos',
            ylabel='Lucro (em $)',
            logy=True, # Escala logarítmica para o lucro
            height=300, width=400, tools=['hover']
        )
        return scatter
    
    # Tabela de Dados
    @pn.depends(year_slider.param.value, genre_select.param.value, min_votes_slider.param.value)
    def data_table(years, genres, min_votes):
        df_filtered = filtered_data(years, genres, min_votes)
        if df_filtered.empty:
            return pn.pane.Markdown("**Sem dados para os filtros selecionados.**")
        
        # Selecionar colunas relevantes para a tabela
        cols = ['title', 'release_year', 'vote_average', 'vote_count', 'budget', 'revenue', 'profit']
        return pn.widgets.DataFrame(df_filtered[cols].head(10), name='Top 10 Filmes Filtrados')
    
    # 4. Layout do Dashboard
    
    controls = pn.Column(
        pn.pane.Markdown("### Controles de Filtro"),
        year_slider,
        genre_select,
        min_votes_slider
    )
    
    plots = pn.Row(
        plot_vote_distribution,
        plot_profit_vs_vote
    )
    
    dashboard = pn.template.MaterialTemplate(
        title='Análise Exploratória de Filmes TMDB',
        sidebar=[controls],
        main=[
            pn.Row(
                pn.Card(plots, title='Visualizações Principais'),
            ),
            pn.Card(data_table, title='Dados Filtrados (Top 10)')
        ]
    )
    
    # Para correr o dashboard no Jupyter Notebook
    dashboard.servable()
    
else:
    pn.pane.Markdown("**Não foi possível carregar os dados para o dashboard.**").servable()


Erro: O ficheiro TMDB_movie_dataset_v11 não foi encontrado. Certifique-se de que está no diretório correto.


In [2]:
# Para visualizar o dashboard, execute esta célula e depois abra o link fornecido pelo Panel.